In [ ]:
%%bash
pip install pandas-profiling

In [1]:
import os
from glob import glob
import pandas as pd
from pandas_profiling import ProfileReport
import csv
from config import REGION, BUCKET, PROJECT, DELIM, RAW_DATA_COLS, RENAMED_COLS

%matplotlib inline
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

# Cloud Setup
This section is only required if running on cloud

In [ ]:
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [ ]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

# Data Profiling
Data profiling is done to better understand the data, and to see if there are any invalid data (e.g. out of bounds data, unexpected data types). No data preprocessing should be done here; it should be done in tf.transform so as to have a consistent data pipeline.

In [2]:
files = glob('data/raw/Raw Data for COA classification_einv*.csv')
df = pd.concat([pd.read_csv(f, usecols=RAW_DATA_COLS) for f in files], ignore_index=True)
df.columns = RENAMED_COLS
df

,fy,business_unit,acc_code,acc_descr,voucher_id,voucher_descr,voucher_origin,vendor_id,vendor_name,voucher_line,voucher_line_descr,voucher_line_long_descr,payment_voucher_amt
0,2016,BU01,123456,DESCR 123456,V00950100,lorem ipsum dolor sit amet,EINV,VDNR2,VDNR2,6,some other text,nan,3453.0
1,2015,BU02,123457,DESCR 123457,V00741138,some text,EINV,VDNR2,VDNR2,4,some text,nan,6375.0
2,2016,BU02,123456,DESCR 123456,V00338789,hello hello,EINV,VDNR9,VDNR9,9,some text,nan,9870.1
3,2017,BU03,123457,DESCR 123457,V0026832,lorem ipsum dolor sit amet,EINV,VDNR8,VDNR8,1,hello world,nan,2868.7
4,2016,BU01,123456,DESCR 123456,V00497887,hello world,EINV,VDNR5,VDNR5,9,hello world,nan,8505.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2015,BU03,123456,DESCR 123456,V00844878,some other text,EINV,VDNR3,VDNR3,8,some text,nan,5852.1
96,2015,BU03,123457,DESCR 123457,V00865190,some text,EINV,VDNR4,VDNR4,8,some text,nan,8000.1
97,2015,BU02,123458,DESCR 123458,V00560899,hello hello,EINV,VDNR3,VDNR3,8,hello world,nan,2356.3
98,2017,BU01,123457,DESCR 123457,V00660038,hello world,EINV,VDNR4,VDNR4,4,some other text,nan,4389.4


In [3]:
ProfileReport(df)

Number of variables,13
Number of observations,100
Total Missing (%),0.0%
Total size in memory,10.2 KiB
Average record size in memory,104.0 B
Numeric,4
Categorical,6
Boolean,0
Date,0
Text (Unique),1
Rejected,2


# Split Data
Example uses 80-10-10 split for train, eval and test - change if necessary

In [4]:
RANDOM_SEED = 42
train_df = df.sample(frac=0.8, random_state=RANDOM_SEED)
eval_df = df.drop(train_df.index)
test_df = eval_df.sample(frac=0.5, random_state=RANDOM_SEED)
eval_df = eval_df.drop(test_df.index)

In [5]:
def export_datasets(on_cloud=False):
    if on_cloud:
        data_dir = 'gs://{bucket}/spam-classification/data/split'.format(bucket=BUCKET)
    else:
        data_dir = 'data/split'
    
    if not on_cloud:
        if not os.path.exists('data'):
            os.mkdir('data')
        if not os.path.exists('data/split'):
            os.mkdir('data/split')
        
    def export_df(df, filename):
        mycsv = df.astype(str).apply(lambda x: DELIM.join(x), axis=1)
        mycsv = mycsv.rename(DELIM.join(df.columns))
        mycsv.to_csv(
            os.path.join(data_dir, filename), 
            header=False, 
            index=False,
            encoding='utf-8'
        )
    
    export_df(train_df, 'train.csv')
    export_df(eval_df, 'eval.csv')
    export_df(test_df, 'test.csv')
  
    return
  
export_datasets(on_cloud=False)